In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import tensorflow as tf
import math
import tensorflow_addons as tfa
import random
import re
import csv

from tensorflow.keras.models import Model
from sklearn import metrics
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from mlxtend.evaluate import mcnemar_table
from statsmodels.stats.contingency_tables import mcnemar




from utils import (
    F1Score,
    plot_metrics,
    plot_accuracy,
    study_oriented_transformation,
    write_csv,
    prediction_results,
    plot_confusion_matrix,
    plot_contigency_table,
)


In [ ]:
# To Activate GPU if there is
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


2022-11-20 18:55:00.251408: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-20 18:55:00.299828: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-20 18:55:00.300643: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [ ]:
SEED = 1037

os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)


In [ ]:
METRICS = [ 
    tf.keras.metrics.BinaryAccuracy(),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall"),
    tfa.metrics.CohenKappa(name="cohen_kappa", num_classes=2),
    F1Score(name="f1_score"),
]

STUDY_TYPES = [
    'XR_ELBOW',
    'XR_FINGER',
    'XR_FOREARM',
    'XR_HAND',
    'XR_HUMERUS',
    'XR_SHOULDER',
    'XR_WRIST',
]

CLASSES = ['NORMAL', 'ABNORMAL']

2022-11-20 18:55:00.612850: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-20 18:55:00.617796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-20 18:55:00.618894: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-20 18:55:00.619837: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

In [ ]:
base_model = 'EfficientnetV2-S'

In [ ]:
data_directory = 'dataset'
test_img = pd.read_csv(os.path.join(data_directory, 'MURA-v1.1_mod/test_image_paths.csv'), names=['path'])

In [ ]:
test_img['label'] = test_img['path'].map(
    lambda x: '1' if 'positive' in x else '0'
)

test_img['study_type'] = test_img['path'].map(
    lambda x: x.split('/')[2]
)

test_img['study'] = test_img['path'].map(
    lambda x: x.split("/")[4]
)

test_img['study_path'] = test_img['path'].map(
    lambda x: re.sub(r"image\d+.png", "", x)
)
test_img

,path,label,study_type,study,study_path
0,MURA-v1.1_mod/test/XR_WRIST/patient11185/study...,1,XR_WRIST,study1_positive,MURA-v1.1_mod/test/XR_WRIST/patient11185/study...
1,MURA-v1.1_mod/test/XR_WRIST/patient11185/study...,1,XR_WRIST,study1_positive,MURA-v1.1_mod/test/XR_WRIST/patient11185/study...
2,MURA-v1.1_mod/test/XR_WRIST/patient11185/study...,1,XR_WRIST,study1_positive,MURA-v1.1_mod/test/XR_WRIST/patient11185/study...
3,MURA-v1.1_mod/test/XR_WRIST/patient11185/study...,1,XR_WRIST,study1_positive,MURA-v1.1_mod/test/XR_WRIST/patient11185/study...
4,MURA-v1.1_mod/test/XR_WRIST/patient11186/study...,1,XR_WRIST,study1_positive,MURA-v1.1_mod/test/XR_WRIST/patient11186/study...
...,...,...,...,...,...
3192,MURA-v1.1_mod/test/XR_FINGER/patient11967/stud...,0,XR_FINGER,study1_negative,MURA-v1.1_mod/test/XR_FINGER/patient11967/stud...
3193,MURA-v1.1_mod/test/XR_FINGER/patient11967/stud...,0,XR_FINGER,study1_negative,MURA-v1.1_mod/test/XR_FINGER/patient11967/stud...
3194,MURA-v1.1_mod/test/XR_FINGER/patient11738/stud...,0,XR_FINGER,study1_negative,MURA-v1.1_mod/test/XR_FINGER/patient11738/stud...
3195,MURA-v1.1_mod/test/XR_FINGER/patient11738/stud...,0,XR_FINGER,study1_negative,MURA-v1.1_mod/test/XR_FINGER/patient11738/stud...


In [ ]:
img_height = img_width = 380

def resize_img(img):
    try:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    except:
        print('error in resizing')
        img1 = isinstance(img, type(None))
        print('Does image is none: ', img1)
        print(img.shape)
    return cv2.resize(img, (img_height, img_width))

def canny_cropping(img):
    convert_img = np.array(img, dtype=np.uint8)

    gray = cv2.cvtColor(convert_img, cv2.COLOR_RGB2GRAY)


    ave_brightness = math.floor(np.average(gray))
    min_pixel = min(gray.flatten())

    edges = cv2.Canny(gray, min_pixel, ave_brightness)
    cnts = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    for c in cnts:
        x, y, w, h = cv2.boundingRect(edges)
        gray = gray[y:y+h, x:x+w]
        break

    return gray

def apply_clahe(img):
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
    return clahe.apply(img.astype(np.uint8))

def preprocessing_without_clahe(img):
    cropped = canny_cropping(img)
    return resize_img(cropped)

def preprocessing_with_clahe(img):
    cropped = canny_cropping(img)
    clahe = apply_clahe(cropped)
    return resize_img(clahe)


# Evaluation of Model Without CLAHE

In [ ]:
batch = 8
data_path = 'dataset/'

test_batches_without_clahe = ImageDataGenerator(
    preprocessing_function=preprocessing_without_clahe
).flow_from_dataframe(
    target_size=(img_height, img_width),
    dataframe=test_img, 
    directory= data_path,
    class_mode='binary',
    x_col='path',
    y_col='label',
    batch_size=batch, 
    shuffle=False)

Found 3197 validated image filenames belonging to 2 classes.


In [ ]:
model_without_clahe = tf.keras.models.load_model(
    'models/without_clahe/efficientnetv2-S_sgd_std_lr_finetuned.h5',
    custom_objects={'F1Score': F1Score}
)

In [ ]:
predictions_without_clahe = model_without_clahe.predict(test_batches_without_clahe, verbose=1)

2022-11-20 18:55:13.972837: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-11-20 18:55:15.648091: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


  2/400 [..............................] - ETA: 22s    

2022-11-20 18:55:17.677960: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


400/400 [==============================] - 63s 134ms/step


In [ ]:
conv_prediction_without_clahe = test_img.copy()

conv_prediction_without_clahe['label'] = conv_prediction_without_clahe['label'].map(int)
conv_prediction_without_clahe['prediction'] = predictions_without_clahe.ravel()

In [ ]:
conv_prediction_without_clahe = pd.DataFrame(
    [*study_oriented_transformation(conv_prediction_without_clahe)],
    columns=['study_type', 'study', 'label', 'prediction'],
)
conv_prediction_without_clahe

,study_type,study,label,prediction
0,XR_ELBOW,MURA-v1.1_mod/test/XR_ELBOW/patient11186/study...,1,1
1,XR_ELBOW,MURA-v1.1_mod/test/XR_ELBOW/patient11189/study...,1,1
2,XR_ELBOW,MURA-v1.1_mod/test/XR_ELBOW/patient11204/study...,0,0
3,XR_ELBOW,MURA-v1.1_mod/test/XR_ELBOW/patient11205/study...,0,0
4,XR_ELBOW,MURA-v1.1_mod/test/XR_ELBOW/patient11217/study...,0,0
...,...,...,...,...
1194,XR_WRIST,MURA-v1.1_mod/test/XR_WRIST/patient11387/study...,0,0
1195,XR_WRIST,MURA-v1.1_mod/test/XR_WRIST/patient11388/study...,0,0
1196,XR_WRIST,MURA-v1.1_mod/test/XR_WRIST/patient11389/study...,0,0
1197,XR_WRIST,MURA-v1.1_mod/test/XR_WRIST/patient11390/study...,0,0


## Evaluation for all body parts concerned

In [ ]:
results_without_clahe = prediction_results(conv_prediction_without_clahe)

write_csv(
    results_without_clahe, 
    f'testing_results/without_clahe/general_prediction_{base_model}_without_clahe.csv'
)

print('=' * 52)
print('Prediction for all Musculoskeletal radiographs')
print('-----------------')
for result in results_without_clahe[:-1]:
    print(f"{result['metric'] + ' ' * (30 - len(result['metric']))}: {result['value']}")
print('=' * 52)


Prediction for all Musculoskeletal radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.6945979289902938
F1 Score                      : 0.8222442899702085
Accuracy                      : 0.8507089241034195
Precision                     : 0.8827292110874201
Recall                        : 0.7695167286245354


In [ ]:
cm_general_without_clahe = results_without_clahe[-1]['value']
print(cm_general_without_clahe)
plot_confusion_matrix(
    cm_general_without_clahe, 
    CLASSES, 
    f'MURA Testing Confusion Matrix for {base_model}Without CLAHE',
    False,
)

[[606  55]
 [124 414]]


<Figure size 432x288 with 0 Axes>

## Evaluation per body types

In [ ]:
for body_part in STUDY_TYPES:
    parts = conv_prediction_without_clahe[conv_prediction_without_clahe['study_type'] == body_part]
    results = prediction_results(parts)
    write_csv(
        results,
        f'testing_results/without_clahe/{body_part}_{base_model}_without_clahe.csv'
    ) 
    parts_cm = results[-1]['value']
    plot_confusion_matrix(
        parts_cm, 
        CLASSES, 
        f'{body_part} Confusion Matrix for efficientnetv2-S Without CLAHE', 
        False
    )
    print('=' * 52)
    print(f'Prediction for {body_part} radiographs')
    print('-----------------')
    for result in results[:-1]:
        print(f"{result['metric'] + ' ' * (30 - len(result['metric']))}: {result['value']}")
    print('=' * 52)


Prediction for XR_ELBOW radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.6935402260077584
F1 Score                      : 0.8099173553719009
Accuracy                      : 0.8544303797468354
Precision                     : 0.8909090909090909
Recall                        : 0.7424242424242424
Prediction for XR_FINGER radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.6529613960867267
F1 Score                      : 0.7999999999999999
Accuracy                      : 0.8285714285714286
Precision                     : 0.8955223880597015
Recall                        : 0.7228915662650602
Prediction for XR_FOREARM radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.7111009488967646
F1 Score                      : 0.8288288288288289
Accuracy                      : 0.8571428571428571
Precision                     : 0.9787234042553191
Recall                        : 0.71875
Prediction for XR_HAND radiographs
-----------------
Cohen's kappa Coefficient (κ

<Figure size 432x288 with 0 Axes>

# Evaluation of Model with CLAHE

In [ ]:
batch = 8
data_path = 'dataset/'

test_batches_with_clahe = ImageDataGenerator(
    preprocessing_function=preprocessing_with_clahe
).flow_from_dataframe(
    target_size=(img_height, img_width),
    dataframe=test_img, 
    directory= data_path,
    class_mode='binary',
    x_col='path',
    y_col='label',
    batch_size=batch, 
    shuffle=False)

Found 3197 validated image filenames belonging to 2 classes.


In [ ]:
model_with_clahe = tf.keras.models.load_model(
    'models/with_clahe/efficientnetv2-S_sgd_std_lr_finetuned_v2.h5',
    custom_objects={'F1Score': F1Score}
)

In [ ]:
predictions_with_clahe = model_with_clahe.predict(test_batches_with_clahe, verbose=1)

400/400 [==============================] - 57s 135ms/step


In [ ]:
conv_prediction_with_clahe = test_img.copy()

conv_prediction_with_clahe['label'] = conv_prediction_with_clahe['label'].map(int)
conv_prediction_with_clahe['prediction'] = predictions_with_clahe.ravel()

In [ ]:
conv_prediction_with_clahe = pd.DataFrame(
    [*study_oriented_transformation(conv_prediction_with_clahe)],
    columns=['study_type', 'study', 'label', 'prediction'],
)

In [ ]:
results_with_clahe = prediction_results(conv_prediction_with_clahe)

write_csv(
    results_with_clahe, 
    f'testing_results/with_clahe/general_prediction_{base_model}_with_clahe.csv'
)

print('=' * 52)
print('Prediction for all Musculoskeletal radiographs')
print('-' * 52)
for result in results_with_clahe[:-1]:
    print(f"{result['metric'] + ' ' * (30 - len(result['metric']))}: {result['value']}")
print('=' * 52)


Prediction for all Musculoskeletal radiographs
----------------------------------------------------
Cohen's kappa Coefficient (κ) : 0.7003980986671361
F1 Score                      : 0.8274509803921568
Accuracy                      : 0.8532110091743119
Precision                     : 0.8755186721991701
Recall                        : 0.7843866171003717


In [ ]:
cm_general_with_clahe = results_with_clahe[-1]['value']
print(cm_general_with_clahe)
plot_confusion_matrix(
    cm_general_with_clahe, 
    CLASSES, 
    f'MURA Testing Confusion Matrix for {base_model} With CLAHE',
    True,
    )

[[601  60]
 [116 422]]


<Figure size 432x288 with 0 Axes>

In [ ]:
for body_part in STUDY_TYPES:
    parts = conv_prediction_with_clahe[conv_prediction_with_clahe['study_type'] == body_part]
    results = prediction_results(parts)
    write_csv(
        results, 
        f'testing_results/with_clahe/{body_part}_prediction_results_{base_model}_with_clahe.csv'
    )
    parts_cm = results[-1]['value']
    plot_confusion_matrix(
        parts_cm, 
        CLASSES, 
        f'{body_part} Confusion Matrix for {base_model} With CLAHE', 
        True
    )
    print('=' * 52)
    print(f'Prediction for {body_part} radiographs')
    print('-----------------')
    for result in results[:-1]:
        print(f"{result['metric'] + ' ' * (30 - len(result['metric']))}: {result['value']}")
    print('=' * 52)

Prediction for XR_ELBOW radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.7226216351780639
F1 Score                      : 0.832
Accuracy                      : 0.8670886075949367
Precision                     : 0.8813559322033898
Recall                        : 0.7878787878787878
Prediction for XR_FINGER radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.6080885258859176
F1 Score                      : 0.7820512820512822
Accuracy                      : 0.8057142857142857
Precision                     : 0.8356164383561644
Recall                        : 0.7349397590361446
Prediction for XR_FOREARM radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.6964163433006163
F1 Score                      : 0.8245614035087719
Accuracy                      : 0.849624060150376
Precision                     : 0.94
Recall                        : 0.734375
Prediction for XR_HAND radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.5722062119756643
F1 S

<Figure size 432x288 with 0 Axes>

# Compare the two models using McNemar's Test

In [ ]:
comparison = mcnemar_table(
    y_target= np.array(conv_prediction_without_clahe['label']),
    y_model1= np.array(conv_prediction_without_clahe['prediction']),
    y_model2= np.array(conv_prediction_with_clahe['prediction']),
)
print(comparison)
plot_contigency_table(
    comparison, 
    ['Correct', 'Wrong'],
    'Contigency Table Between {base_model} Without and With CLAHE',
    f'{base_model}',
)

[[988  32]
 [ 35 144]]


<Figure size 432x288 with 0 Axes>

In [ ]:
mncnemar_without_correction = mcnemar(comparison, exact=False)
mncnemar_with_correction = mcnemar(comparison, exact=False, correction=True)

print('=' * 35)
print('McNemar\'s test without correction')
print(mncnemar_without_correction)
print('=' * 35)
print('McNemar\'s test with correction')
print(mncnemar_with_correction)

McNemar's test without correction
pvalue      0.8069683671707382
statistic   0.05970149253731343
McNemar's test with correction
pvalue      0.8069683671707382
statistic   0.05970149253731343
